# Neural Network
***
**Name**: Nicholas Varberg
***

## Goal

The goal of this assignment is to build a one-hidden-layer back propagation network to process real data.  For this assignment you will implement the neural net (activation function and training code) yourself, not using tensorflow or other deep learning frameworks. The purpose is for you to understand the nitty gritty of what these frameworks are doing for you before we switch over to using them.

## Data 
***

In this assignment you will be using the Occupancy Detection data set (ODD). It consists of experimental data used for binary classification of room occupancy (i.e., room is occupied versus empty) based on temperature, humidity, light, and CO2 sensors. The train and test data sets are each collected over a week period.

The data set includes time stamps with date and hour/minute/second within the day. You are **not to use time stamp features** for predicting occupancy. Since this is a commercial office building, the time stamp is a strong predictor of occupancy. Rather, the goal is to determine whether occupancy - **occ** can be sensed from:
1. temperature, expressed in degrees Celsius - **temp**
2. relative humidity, expressed as a % - **hum**
3. light, in lux - **light**
4. CO2, in ppm - **cdx**
5. humidity ratio, which is derived from the temperature and the relative humidity - **hum_ratio**

In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('data/datatraining.txt')
X = data[['date', 'Humidity', 'Light', 'CO2', 'HumidityRatio', 'Temperature']].values
y = data['Occupancy'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

### 1. Perceptron
***

Using the perceptron code you wrote for Assignment 1, train a perceptron (linear activation function with a binary threshold) using the training set. Your perceptron should have the 5 input variables described above.

**Part A**: Report the training and test set performance in terms of % examples classified correctly after 100 epochs.

**NOTE**: The Perceptron Update Rule is guaranteed to converge only if there is a setting of the weights that will classify the training set perfectly.  (The learning rule minimizes mistakes. When all examples are classified correctly, the weights stop changing.)  With a noisy data set like this one, the algorithm may not terminate.  Also remember that the perceptron algorithm is performing stochastic gradient descent. Thus, it will jitter around the ideal solution continually changing the weights from one iteration to the next. The weight changes will have a small effect on performance, so you'll see training set performance jitter a bit as well.

In [20]:
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, accuracy_score

def perceptron_update(X, y, w, alpha):
    """One epoch of perceptron updates.
    Args:
      X: Training features
      y: Training labels
      w: Weights
      alpha: Stepsize 
      
    Returns:
      w: Updated weight vector
      incorrect: Number of incorrectly classified 
        examples using the parameter values at 
        the end of the epoch
    """
    # Shuffling the indices is working properly
    shuffled_i = shuffle(range(len(X)))
    for i in shuffled_i:
        a = np.dot(X[i], w)
        pred = 0
        if a >= 0:
            pred = 1
        if pred != y[i]:
            w = w + alpha*(y[i] - pred)*X[i]
    
    incorrect = None
    # Change and score binary predictions
    y_pred = []
    for pred in np.dot(X,w):
        if pred < 0:
            y_pred.append(0)
        else:
            y_pred.append(1)
    # After epoch count number incorrect
    incorrect = len(y) - accuracy_score(y,np.array(y_pred), normalize=False)
    return w, incorrect

def perceptron(X, y, maxEpochs, alpha):
    # Add array of ones to X so that intercept (b) is solved as well
    b_col   = np.array([1]*len(X)).reshape(-1,1)
    X_b     = np.append(X, b_col, axis=1)
    
    w, incorrect = np.ones(len(X_b[0])), None
    for i in range(maxEpochs):
        w,incorrect = perceptron_update(X_b, y, w, alpha)
    
    return w, incorrect

w, n = perceptron(X_train[:,1:], y_train, 100, 0.001)

print(w)
print(n)

[-15.092387750001368 3.5969833333335335 0.7124666666659011
 0.9959162440509737 -69.21180966666331 -1.9359999999998985]
129


Resporting Train and Test Accucray %

In [29]:
print('Train accuracy % after 100 epochs', round((1 - n/len(y_train))*100, 1))

# Add array of ones to X so that intercept (b) is solved as well
b_col   = np.array([1]*len(X_test[:,1:])).reshape(-1,1)
X_b     = np.append(X_test[:,1:], b_col, axis=1)
#Score test set
y_pred = []
for pred in np.dot(X_b,w):
    if pred < 0:
        y_pred.append(0)
    else:
        y_pred.append(1)
# After epoch count number incorrect
test_acc = round(accuracy_score(y_test,np.array(y_pred), normalize=False)/len(y_test)*100, 1)
print('Test accuracy % after 100 epochs', test_acc)

Train accuracy % after 100 epochs 98.2
Test accuracy % after 100 epochs 98.5


### 2. 1-layer feedforward neural net
***

In this part, you will implement a 1-layer feedforward neural network from scratch as shown in the figure below.
<img src="./res/mlp_ann.png" alt="mlp_ann" style="width:500px;"/>
Your tasks are as follows:

**PART 2.A**

1. Implement `sigmoid(x)`: In this assignment, we will be using the sigmoid activation function as the non-linearity.
2. Implement `forward(x)` which calculates the activations at layer 1, 2 and 3 (stored in variables `_a01`, `_a12`, `_a23` respectively) and returns the final layer activations.
3. Implement `backward(y, y_hat)` such that it returns layer 1 and layer 2 gradients using the mean squared loss between the predicted and groundtruth vectors.
4. Implement `update(l1_grad, l2_grad)` function which will update the weights and biases stored in `_W1, _W2 and _b1, _b2`.
5. Implement`train(X, y, epochs)`.
6. Implement `predict(X)` to return one-hot encoded representation of the `_a23` layer activation.

In [254]:

rndg = np.random.RandomState(seed=0)

class MLP(object):
    """
        -- Skeleton Code for a Multi-Layer Perceptron Neural Net.
           Network has 1 input layer followed by a hidden layer and
           an output layer. 
        
    """
    def __init__(self, n_vis=5, n_hid=10, n_out=1, batch_size=7, lr=0.1):
        """
            Initialize an MLP object.
            
            params
            
            n_vis (int): # of neurons in input layer.
            n_hid (int): # of neurons in hidden layer.
            n_out (int): # number of neurons in final output layer.
            batch_size (int): # number of X,y instances in a mini-batch.
            lr (double): learning rate/step size.

            class variables:
            
            _W1, _b1 (np.ndarray): 
                Weights and biases of the input layer respectively.
            
            _W2, _b2 (np.ndarray):
                Weights and biases of the hidden layer respectively.
            
            _a01, _a12, _a23 (np.ndarray or None):
                Activations from input, hidden and output layer respectively. 
                
            _l1_grad_b1, _l2_grad_b2 (np.ndarray or None):
                Gradients of the loss w.r.t. the biases. 
        """
        self._n_vis = n_vis
        self._n_hid = n_hid
        self._n_out = n_out
        
        self._W1 = rndg.normal(size=(self._n_hid, self._n_vis))
        self._b1 = np.ones((self._n_hid, 1))

        self._W2 = rndg.normal(size=(self._n_out, self._n_hid))
        self._b2 = np.ones((self._n_out, 1))

        self._a01 = None
        self._a12 = None
        self._a23 = None
        self._z12 = None
        self._z23 = None
        
        self._l1_grad_b1 = None
        self._l2_grad_b2 = None
        
        self.batch_size = batch_size
        self.lr = lr

    def sigmoid(self, X):
        """
            Sigmoid Logistic Function.
            
            params:
            
            X (np.ndarray)
            
            return:
            
            result (np.ndarray): result = f(X) 
                where f is the sigmoid function.
            
        """
        return 1/(1 + np.exp(-X))
        

    def forward(self, X):
        """
            Feed forward an input through each 
            of the layers of network.
            
            Store each layer activation in the class 
            variables defined in the constructor. You'll
            need them later during backpropogation.
            
            params:
            
            X (np.ndarray): batch_size x 5 dimensional array
            
            return:
            
            y_hat (np.ndarray): 1 x batch_size dimensional array
                representing final layer outputs.
            
        """
        # First layer is just input
        print('------start forward()-----')
        if self._a01 is None:
            #print('_a01 was none')
            self._a01 = X
        else:
            #print('_a01 was not none')
            self._a01 = X
        #print(self._a01.shape, '_a01 shape')
        #print('--end layer 1--')
        
        # Second layer is linear transformation then sigmoid
        #print(self._W1.T.shape, 'W1.T shape')
        z = self._a01 @ self._W1.T
        #print(z.shape, 'z shape')
        #print(self._b1.T.shape, 'b1.T shape')
        z = z + self._b1.T
        self._z12 = z
        #print(z.shape, 'z shape')
        if self._a12 is None:
            #print('_a12 was none')
            self._a12 = self.sigmoid(z)
        else:
            #print('_a12 was not none')
            self._a12 = self.sigmoid(z)
        #print(self._a12.shape, 'self._a12 shape')
        #print('--end layer 2--')
            
        # Third layer (output) is linear then sigmoid of previous hidden layer
        #print(self._W2.T.shape, 'W2.T shape')
        z = self._a12 @ self._W2.T
        #print(z.shape, 'z shape')
        #print(self._b2.shape, 'b2 shape')
        z = (z + self._b2).reshape(1,-1)
        self._z23 = z
        #print(z.shape, ' z shape')
        if self._a23 is None:
            #print('_a23 was none')
            self._a23 = self.sigmoid(z)
        else:
            #print('_a23 not none')
            self._a23 = self.sigmoid(z)
        #print(self._a23.shape, '_a23 shape')
        print('------done with forward()-----')
        return self._a23

    def backward2(self, y, y_hat):
        
        """
            Implement the backpropogation algorithm.
            Assume mean squared loss at the output.
            calculate and update gradients of the loss w.r.t. the biases
            
            params:
            
            y (np.ndarray): batch_size x 1 dimensional vector
                of grounf truth labels.
            y_hat (np.nd_array): 1 x 7 dimensional vector
                of predicted lables from forward().
            
            return:
            
            l1_grad (np.ndarray): n_hid x n_vis dimensional array
                representing gradients for hidden layer. (gradients of the loss w.r.t. the _W1)
                
            l2_grad (np.ndarray): 1 x n_hid dimensional array
                representing gradients for input layer. (gradients of the loss w.r.t. the _W2)
            
        """
        # Compute original delta
        print('------start backward()-----')
        dl_da = np.mean(y - y_hat)
        print(dl_da.shape, 'dl_da.shape')
        # element wise multiplication
        g_prime_z = self.sigmoid(self._z23)*(1- self.sigmoid(self._z23))
        print(g_prime_z.shape, 'g_prime_z23 shape')
        delta2 = dl_da * g_prime_z
        #delta2 = np.mean(delta2)
        #print(delta2.shape, 'delta2 shape')
        
        # For l = 2
        self._l2_grad_b2 = delta2
        print(self._l2_grad_b2.shape, '_l2_grad_b2 shape')
        print(self._b1.shape, 'should be _l2_grad_b2 shape')
        print(delta2.shape, 'delta2 shape')
        print(self._a12.shape, '_a12 shape')
        l2_grad = delta2 @ self._a12
        print(l2_grad.shape, 'l2_grad shape')
        print(1, 'x', self._n_hid, 'should be l2_grad shape')
        print(self._W2.shape, '_W2 shape should = l2_grad shape')
        
        # For l = 1
        print(self._W2.T.shape, '_W2.T shape')
        delta1 = self._W2.T @ delta2
        print(delta1.T.shape, 'delta1 shape')
        delta1 = delta1.T * self.sigmoid(self._z12)*(1- self.sigmoid(self._z12))
        self._l1_grad_b1 = delta1
        print(delta1.shape, 'delta1 shape')
        print(self._a01.shape, '_a01 shape')
        l1_grad = delta1.T @ self._a01
        print(l1_grad.shape, 'l1_grad shape')
        print(self._n_hid,'x', self._n_vis, 'should be l1_grad shape')
        print(self._W1.shape, '_W1 shape should = l1_grad shape')
        print('------done with backward()-----')
        #return l1_grad, l2_grad
        
    def backward(self, y, y_hat):
        # Compute original delta
        print('------start backward()-----')
        dW2 = (np.mean(y - y_hat) * self._a23*(1 - self._a23)).reshape(1,-1) @ self._a12
        print(dW2.shape, 'dW2', 'should be', self._W2.shape)
        l2_grad = dW2
        db2 = np.mean(np.mean(y - y_hat) * self._a23*(1 - self._a23), axis=1)
        #db22 = np.mean((y - y_hat) * self._a23*(1 - self._a23), axis=1)
        #print(db22)
        print(db2.shape,'db2','should be', self._b2.shape)
        self._l1_grad_b2 = db2
        
        a = (np.mean(y - y_hat) * self._a23*(1-self._a23))
        print(a.shape, 'a')
        b = self._W2.T @ a
        print(b.shape, 'b')
        c = b.T * self._a12*(1-self._a12)
        print(self._a01.shape, '_a01', c.shape, 'c')
        d = c.T @ self._a01
        dW1 = d
        print(dW1.shape, 'dW1', 'should be', self._W1.shape)
        l1_grad = dW1
        db1 = np.mean(c.T, axis = 1)
        print(db1.shape,'db1','should be', self._b1.shape)
        self._l1_grad_b1 = db1
        print('------done with backward()-----')
        return l1_grad, l2_grad
    
    def update(self, l1_grad, l2_grad):
        """
            Implement the update rule for network weights and biases.
            
            params:
            
            l1_grad (np.ndarray): gradients for input layer.
            l2_grad (np.ndarray): gradients for hidden layer.
            
            return:
            
            none.
        """
        self._W1 = self._W1 - l1_grad
        self._b1 = self._b1 - np.sum(self._l1_grad_b1, axis=0)
        self._W2 = self._W2 - l2_grad
        self._b2 = self._b2 - self._l1_grad_b1
        
    def predict(self, X, threshold = 0.5):
        """
            Returns one hot encoding of vector X using 0.5
            as the threshold.
            
            params:
            
            X (np.ndarray): predictions/activations of the output layer.
            
            return:
            
            y (np.ndarray): one hot encoding of output layer.
        """
        y = np.zeros(len(X))
        for i in len(X):
            if X[i] > threshold:
                y[i] = 1
        
        return y
                
    
    def train(self, X, y, epochs):
        """
            Implement the train loop for n epochs.
            In each epoch do the following:
            1. Shuffle the dataset.
            2. create self.batch_size sized-mini-batches of the dataset.
            3. get network predictions using forward().
            4. calculate the gradients using backward().
            5. update the network weights using update().
            6. Repeat 1-5 until convergence.
            
            params:
            
            X (np.ndarray): N x 5 dimensional ndarray of inputs.
            y (np.ndarray): N x 1 dimensional array of true labels.
            epochs (int): # of epochs to train the network for. 
            
            return:
            
            none.
        """
        for ep in range(epochs):
            
            # 1. Shuffle the dataset
            shuffled_i = shuffle(range(len(X)))
            X, y       = X[shuffled_i], y[shuffled_i]
            
            
            # 2. Create mini batches
            for i in range(0, len(X), self.batch_size):
                X_batch, y_batch = X[i:i+self.batch_size], y[i:i+self.batch_size]
                
                # 3. Get predictions with forward()
                y_hat_batch = self.forward(X_batch)
                
                # 4. Calculate gradients with backward()
                l1_grad, l2_grad = self.backward(y_batch, y_hat_batch)
                
                # 5. Update the network with update()
                self.update(l1_grad, l2_grad)
            
    def test(self, X_test, y_test):
        """
            Implement the test function which reports accuracy of 
            the model on the test set X_test against the true labels
            y_test.
            
            You may re-use the predict and accuracy functions defined above.
            
            params:
            
            X_test (np.ndarray): N x M dimensional array where M is the 
                number of attributes considered for training the model.
            y_test (np.ndarray): N x 1 dimensional array.
            
            return:
            
            accuracy (double): accuracy of the predicted labels against the 
                groundtruth labels.
        """
        
        raise NotImplementedError()
    
    @classmethod
    def accuracy(self, y, y_hat):
        len_ = y.flatten().shape[0]
        return np.sum(y.flatten() == y_hat.flatten())/len_

    

In [255]:
import unittest
from sklearn.datasets import make_classification

def load_params(path, model):
    
    params = pickle.load(open(path, 'rb'))
    for key in params:
        model.__dict__[key] = params[key]

class TestMLP(unittest.TestCase):
    def setUp(self):
        
        self.net = MLP()
        self.pre_trained_model = load_params("./res/mlp.pkl", self.net)
        self.X = np.load("./res/x.npy")
        self.y = np.load("./res/y.npy")
        self.p = np.load("./res/p.npy")
        self.sigX = np.asarray([-1, -2, -3, 0, 1, 2, 3])
    
    def test_forward(self):
        self.assertEqual(np.sum(self.net.forward(self.X[901:])), np.sum(self.p))
    
    def test_train(self):
        x, y = make_classification(
            n_samples=1000,
            n_features=5,
            n_redundant=0,
            random_state=rndg
        )
        net = MLP(5, 5, 1, 7, 0.01)
        net.train(self.X[:900], self.y[:900], 100)
        self.assertTrue(net.accuracy(net.predict(self.X[901:]), self.y[901:]) > 0.88)
    
    def test_sigmoid(self):
        self.assertEqual(
            np.sum(self.net.sigmoid(self.sigX)), 3.5
        )
        
suite = unittest.TestLoader().loadTestsFromTestCase(TestMLP)
unittest.TextTestRunner().run(suite)

..E

------start forward()-----
------done with forward()-----
------start forward()-----
------done with forward()-----
------start backward()-----
(1, 5) dW2 should be (1, 5)
(1,) db2 should be (1, 1)
(1, 7) a
(5, 7) b
(7, 5) _a01 (7, 5) c
(5, 5) dW1 should be (5, 5)
(5,) db1 should be (5, 1)
------done with backward()-----
------start forward()-----
------done with forward()-----
------start backward()-----



ERROR: test_train (__main__.TestMLP)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-255-3580bbb4e978>", line 31, in test_train
    net.train(self.X[:900], self.y[:900], 100)
  File "<ipython-input-254-9e8b11b6456a>", line 304, in train
    l1_grad, l2_grad = self.backward(y_batch, y_hat_batch)
  File "<ipython-input-254-9e8b11b6456a>", line 204, in backward
    dW2 = (np.mean(y - y_hat) * self._a23*(1 - self._a23)).reshape(1,-1) @ self._a12
ValueError: operands could not be broadcast together with shapes (7,) (1,35) 

----------------------------------------------------------------------
Ran 3 tests in 0.026s

FAILED (errors=1)


<unittest.runner.TextTestResult run=3 errors=1 failures=0>

**PART 2.B**

Report the baseline performance of the untrained MLP that you built in **PART 2.A** using the selected values. To calculate the baseline score, feed-forward entire test set through the test function. The test function will return a set of predictions. Calculate the accuracy of these predictions against ground truth training lables.  
Report:
1. accuracy

In [6]:
### Part 2.B

**PART 2.C**

Select a learning rate, batch size and train the network for 100 epochs. Plot a graph of network's accuracy on the test data as a function of epoch. On the same graph, plot a constant horizontal line of the baseline score that you reported in **PART 2.B**.  

Report:
1. learning rate
2. batch size

In [7]:
### Part 2.C

**PART 2.D**

Now train nets with varying size of the hidden layer $H={1, 2, 5, 10..}$ for max epochs = 100. Make a plot of the nets' accuracy on test set as a function of $H$.

In [8]:
### Part 2.D

### 3. Extra Credit

See how much adding information about time of day helps the network. Add a new set of inputs that represent the time of day. (Don't add information about day of week or absolute date.)  


**PART 3.A**  

Determine an appropriate representation for the time of day. Describe the representation you used. For example, you might add one unit with a value ranging from 0 to 1 for times ranging from 00:00 to 23:59. Report the representation you selected.


In [9]:
### Part 3.A

**PART 3.B**  

Train your net with $H=5$ hidden and compare training and test set performance to the net you built in (2c)

In [10]:
### Part 3.B